In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

trade = pd.read_csv('trade_train.csv', index_col=0)
stock = pd.read_csv('stocks.csv', index_col=0)
answer = pd.read_csv('answer_sheet.csv')

In [172]:
seq2seq_df = pd.read_csv('seq2seq_df.csv')

In [296]:
seq2seq_df['매수고객수'].quantile(0.999)

131.20100000001548

In [297]:
seq2seq_df['매도고객수'].quantile(0.999)

106.0

In [298]:
y1 = seq2seq_df['매수고객수'].quantile(0.999)
y2 = seq2seq_df['매도고객수'].quantile(0.999)

seq2seq_df.loc[seq2seq_df['매수고객수']>y1] = y1
seq2seq_df.loc[seq2seq_df['매도고객수']>y2] = y2

In [304]:
df = seq2seq_df.copy()

df = df.sort_values(by=['기준년월','종목번호','그룹번호'])

df['매수고객수shift1'] = df.groupby(['종목번호','그룹번호'])['매수고객수'].shift(1, fill_value=0)
df['매수고객수shift2'] = df.groupby(['종목번호','그룹번호'])['매수고객수'].shift(2, fill_value=0)

rolling_cols = ['매도고객수', '매수고객수']
for rolling_col in rolling_cols:
    for i in [2,3]:
        df[str(rolling_col)+'rolling_mean'+str(i)] = \
            df.groupby(['그룹번호','종목번호'])[rolling_col].transform(lambda x : x.rolling(i).mean())
    for i in [2,3]:
        df[str(rolling_col)+'rolling_std'+str(i)] = \
            df.groupby(['그룹번호','종목번호'])[rolling_col].transform(lambda x : x.rolling(i).std())
    for i in [2,3]:
        df[str(rolling_col)+'rolling_max'+str(i)] = \
            df.groupby(['그룹번호','종목번호'])[rolling_col].transform(lambda x : x.rolling(i).max())
    for i in [2,3]:
        df[str(rolling_col)+'rolling_min'+str(i)] = \
            df.groupby(['그룹번호','종목번호'])[rolling_col].transform(lambda x : x.rolling(i).min())
        
diff_cols = ['매도고객수', '매수고객수']
for diff_col in diff_cols:
    df[str(diff_col)+'diff1'] = df.groupby(['그룹번호','종목번호'])[diff_col].diff(1)
    df[str(diff_col)+'diff2'] = df.groupby(['그룹번호','종목번호'])[diff_col].diff(2)
    
df_dropna = df.dropna(axis=0).reset_index(drop=True)

df_dropna['target'] = df_dropna.groupby(['종목번호','그룹번호'])['매수고객수'].shift(-1)

df_cols = df_dropna.columns
for cols in df_cols:
    if df_dropna[cols].dtypes == np.float64:
        df_dropna[cols] = round(df_dropna[cols], 2)

In [305]:
df_dropna.isnull().sum()

종목번호                     0
min                      0
max                      0
mean                     0
std                      0
기준년월                     0
거래금액_만원단위                0
거래량1                     0
거래량2                     0
거래량3                     0
거래량4                     0
시장구분                     0
표준산업구분코드_대분류             0
그룹번호                     0
그룹내고객수                   0
매수고객수                    0
매도고객수                    0
매수고객수shift1              0
매수고객수shift2              0
매도고객수rolling_mean2       0
매도고객수rolling_mean3       0
매도고객수rolling_std2        0
매도고객수rolling_std3        0
매도고객수rolling_max2        0
매도고객수rolling_max3        0
매도고객수rolling_min2        0
매도고객수rolling_min3        0
매수고객수rolling_mean2       0
매수고객수rolling_mean3       0
매수고객수rolling_std2        0
매수고객수rolling_std3        0
매수고객수rolling_max2        0
매수고객수rolling_max3        0
매수고객수rolling_min2        0
매수고객수rolling_min3        0
매도고객수diff1               0
매도고객수diff2               0
매

In [306]:
df_dropna.columns

Index(['종목번호', 'min', 'max', 'mean', 'std', '기준년월', '거래금액_만원단위', '거래량1',
       '거래량2', '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수',
       '매수고객수', '매도고객수', '매수고객수shift1', '매수고객수shift2', '매도고객수rolling_mean2',
       '매도고객수rolling_mean3', '매도고객수rolling_std2', '매도고객수rolling_std3',
       '매도고객수rolling_max2', '매도고객수rolling_max3', '매도고객수rolling_min2',
       '매도고객수rolling_min3', '매수고객수rolling_mean2', '매수고객수rolling_mean3',
       '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2',
       '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'target'],
      dtype='object')

In [307]:
df_dropna.to_csv('df_dropna_deep.csv', index=False)

In [308]:
train_ = [201911, 201912, 202001, 202002, 202003, 202004]
val_ = 202005
test_ = 202006

X_train = df_dropna.loc[df_dropna['기준년월'].isin(train_)].reset_index(drop=True)
X_val = df_dropna.loc[df_dropna['기준년월']==val_].reset_index(drop=True)
X_test = df_dropna.loc[df_dropna['기준년월']==test_].reset_index(drop=True)

X_train['year'] = X_train['기준년월'].map(str).map(lambda x : x[:4])
X_val['year'] = X_val['기준년월'].map(str).map(lambda x : x[:4])
X_test['year'] = X_test['기준년월'].map(str).map(lambda x : x[:4])

X_train.drop('기준년월', axis=1 , inplace=True)
X_val.drop('기준년월', axis=1 , inplace=True)
X_test.drop('기준년월', axis=1 ,inplace=True)

In [309]:
category_cols = ['종목번호', '시장구분', '표준산업구분코드_대분류','그룹번호','year']

In [216]:
X_train.columns

Index(['종목번호', 'min', 'max', 'mean', 'std', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매수고객수',
       '매도고객수', '매수고객수shift1', '매수고객수shift2', '매도고객수rolling_mean2',
       '매도고객수rolling_mean3', '매도고객수rolling_std2', '매도고객수rolling_std3',
       '매도고객수rolling_max2', '매도고객수rolling_max3', '매도고객수rolling_min2',
       '매도고객수rolling_min3', '매수고객수rolling_mean2', '매수고객수rolling_mean3',
       '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2',
       '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'target',
       'year'],
      dtype='object')

In [310]:
import category_encoders as ce
encoder = ce.TargetEncoder()
encoder.fit(X_train.loc[X_train['year']=='2020', category_cols], X_train.loc[X_train['year']=='2020', 'target'])

X_train[category_cols] = encoder.transform(X_train[category_cols])
X_val[category_cols] = encoder.transform(X_val[category_cols])
X_test[category_cols] = encoder.transform(X_test[category_cols])

In [218]:
X_train.columns

Index(['종목번호', 'min', 'max', 'mean', 'std', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매수고객수',
       '매도고객수', '매수고객수shift1', '매수고객수shift2', '매도고객수rolling_mean2',
       '매도고객수rolling_mean3', '매도고객수rolling_std2', '매도고객수rolling_std3',
       '매도고객수rolling_max2', '매도고객수rolling_max3', '매도고객수rolling_min2',
       '매도고객수rolling_min3', '매수고객수rolling_mean2', '매수고객수rolling_mean3',
       '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2',
       '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'target',
       'year'],
      dtype='object')

In [311]:
cols = ['매수고객수shift1', '매수고객수shift2','매수고객수',
       '종목번호', 'min', 'max', 'mean', 'std', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', 
       '그룹번호', '그룹내고객수', '매도고객수',  '매도고객수rolling_mean2',
       '매도고객수rolling_mean3', '매도고객수rolling_std2', '매도고객수rolling_std3',
       '매도고객수rolling_max2', '매도고객수rolling_max3', '매도고객수rolling_min2',
       '매도고객수rolling_min3', '매수고객수rolling_mean2', '매수고객수rolling_mean3',
       '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2',
       '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'year', 'target']

X_train = X_train.loc[:,cols]
X_val = X_val.loc[:,cols]
X_test = X_test.loc[:,cols]

In [220]:
X_train.columns

Index(['매수고객수shift1', '매수고객수shift2', '매수고객수', '종목번호', 'min', 'max', 'mean',
       'std', '거래금액_만원단위', '거래량1', '거래량2', '거래량3', '거래량4', '시장구분',
       '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매도고객수', '매도고객수rolling_mean2',
       '매도고객수rolling_mean3', '매도고객수rolling_std2', '매도고객수rolling_std3',
       '매도고객수rolling_max2', '매도고객수rolling_max3', '매도고객수rolling_min2',
       '매도고객수rolling_min3', '매수고객수rolling_mean2', '매수고객수rolling_mean3',
       '매수고객수rolling_std2', '매수고객수rolling_std3', '매수고객수rolling_max2',
       '매수고객수rolling_max3', '매수고객수rolling_min2', '매수고객수rolling_min3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'year',
       'target'],
      dtype='object')

In [312]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1, 1))
scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

In [225]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.metrics import mean_squared_error

In [226]:
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cuda


In [313]:
class MyDataset(Dataset):
    def __init__(self, one_df):
        super(MyDataset, self).__init__()
        
        self.df = one_df
  
    def __len__(self):
        return self.df.shape[0]
        
    def __getitem__(self, idx):
        rnn_x = self.df.iloc[idx, 0:3].values
        company_x = self.df.iloc[idx, 3:15].values
        group_x = self.df.iloc[idx, 15:-1].values
        y = self.df.iloc[idx, -1]
        return rnn_x.reshape(3,-1), company_x, group_x, y

In [314]:
batch_size=64

In [315]:
train_dataset = MyDataset(X_train_scaled)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = MyDataset(X_val_scaled)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [316]:
print(train_dataset[0][0].shape) # seq_size, input_size
print(train_dataset[0][1].shape)
print(train_dataset[0][2].shape)

(3, 1)
(12,)
(24,)


In [317]:
class DeepModel(nn.Module):
    def __init__(self, rnn_hidden_size=8):
        super(DeepModel, self).__init__()
        
        self.rnn = nn.LSTM(input_size=1, hidden_size=rnn_hidden_size, batch_first=True)

        self.fc1_company = nn.Linear(12, 64) 
        self.fc2_company = nn.Linear(64, 32)
        
        self.fc1_group = nn.Linear(24, 64)
        self.fc2_group = nn.Linear(64, 32)
        
        self.fc1 = nn.Linear(72, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        
        self.batch_size = batch_size
        
    def forward(self, rnn_x, company_x, group_x):
        
        output, (h, c) = self.rnn(rnn_x)
        company_x = F.relu(self.fc1_company(company_x))
        company_x = F.relu(self.fc2_company(company_x))
        group_x = F.relu(self.fc1_group(group_x))
        group_x = F.relu(self.fc2_group(group_x))
        
        x = torch.cat([h.view(h.size(1), -1), company_x, group_x], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [318]:
model = DeepModel().to(device)
criterion = nn.MSELoss()
learning_rate = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [319]:
# train
total_step = len(train_dataloader)
num_epochs = 20
for epoch in range(num_epochs):
    for i, (rnn_x, company_x, group_x, y) in enumerate(train_dataloader):
        rnn_x = rnn_x.float().to(device)
        company_x = company_x.float().to(device)
        group_x = group_x.float().to(device)
        y = y.float().to(device)

        outputs = model(rnn_x, company_x, group_x)
        outputs = outputs.float()
        optimizer.zero_grad()
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        if (i+1) % 250 == 0:
            print(f'Epoch : {epoch+1} / {num_epochs},\
            Step : {i+1} / {total_step}, Loss : {loss.item():.7f}')

C:\Users\ghktj\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch : 1 / 20,            Step : 250 / 597, Loss : 0.0034229
Epoch : 1 / 20,            Step : 500 / 597, Loss : 0.0092131


C:\Users\ghktj\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch : 2 / 20,            Step : 250 / 597, Loss : 0.0116492
Epoch : 2 / 20,            Step : 500 / 597, Loss : 0.0011899
Epoch : 3 / 20,            Step : 250 / 597, Loss : 0.0009811
Epoch : 3 / 20,            Step : 500 / 597, Loss : 0.0011471
Epoch : 4 / 20,            Step : 250 / 597, Loss : 0.0094079
Epoch : 4 / 20,            Step : 500 / 597, Loss : 0.0531990
Epoch : 5 / 20,            Step : 250 / 597, Loss : 0.0620425
Epoch : 5 / 20,            Step : 500 / 597, Loss : 0.0074792
Epoch : 6 / 20,            Step : 250 / 597, Loss : 0.0040300
Epoch : 6 / 20,            Step : 500 / 597, Loss : 0.0017104
Epoch : 7 / 20,            Step : 250 / 597, Loss : 0.0005218
Epoch : 7 / 20,            Step : 500 / 597, Loss : 0.0051581
Epoch : 8 / 20,            Step : 250 / 597, Loss : 0.0050161
Epoch : 8 / 20,            Step : 500 / 597, Loss : 0.0008830
Epoch : 9 / 20,            Step : 250 / 597, Loss : 0.0139184
Epoch : 9 / 20,            Step : 500 / 597, Loss : 0.0106000
Epoch : 

In [320]:
## save checkpoints
checkpoints = {
    'model_state' : model.state_dict(),
    'optim_state' : optimizer.state_dict(),
}
 
FILE = 'checkpoints.pth'
torch.save(checkpoints, FILE)
 
## load checkpoints
# loaded_checkpoints = torch.load(FILE)
# model.load_state_dict(loaded_checkpoints['model_state'])
# optimizer.load_state_dict(loaded_checkpoints['optim_state'])

In [324]:
# val
total_output = []
total_y = []
with torch.no_grad():
    for i, (rnn_x, company_x, group_x, y) in enumerate(val_dataloader):
        rnn_x = rnn_x.float().to(device)
        company_x = company_x.float().to(device)
        group_x = group_x.float().to(device)

        outputs = model(rnn_x, company_x, group_x)
        outputs = outputs.to('cpu')
        total_output = np.append(total_output, outputs)
        total_y = np.append(total_y, y)

X_val_scaled_ = X_val_scaled.copy()        
X_val_scaled_['target']= total_output
X_val_inverse = pd.DataFrame(scaler.inverse_transform(X_val_scaled_), columns=X_train.columns)

In [331]:
mean_squared_error(X_val_inverse['target'],X_val['target'])

88.35010035409161

In [ ]:
print(f'val score : {np.sqrt()}')

In [155]:
# df_dropna = pd.read_csv('df_dropna.csv')
# test_ = 202006

# X_test = df_dropna.loc[df_dropna['기준년월']==test_, :'매수고객수diff2'].reset_index(drop=True)
# X_test['year'] = X_test['기준년월'].map(str).map(lambda x : x[:4])
# X_test.drop('기준년월', axis=1 ,inplace=True)

In [157]:
answer = pd.read_csv('answer_sheet.csv')
result_cols = ['종목번호','그룹번호']
sub = X_test[result_cols]
sub['pred'] = total_output

sub = sub.sort_values(by=['그룹번호','pred'], ascending=[True, False])
group_num = sub['그룹번호'].unique()

for num in group_num:
    val = sub.loc[sub['그룹번호']==num][:3]['종목번호'].sort_values().values
    answer.loc[answer['그룹명']==num,'종목번호1':] = val
answer

<ipython-input-157-1b0028a3666b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['pred'] = total_output


,그룹명,종목번호1,종목번호2,종목번호3
0,MAD01,A000100,A000140,A000440
1,MAD02,A000100,A000140,A000440
2,MAD03,A000100,A000140,A000440
3,MAD04,A000100,A000140,A000440
4,MAD05,A000100,A000140,A000440
5,MAD06,A000100,A000140,A000440
6,MAD07,A000100,A000140,A000440
7,MAD08,A000100,A000140,A000440
8,MAD09,A000100,A000140,A000440
9,MAD10,A000100,A000140,A000440
